In [1]:
import os

In [2]:
pwd

'c:\\Users\\PUJA\\OneDrive\\Desktop\\Machine_Learning_Project\\ML_project-_using_MlFlow\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'c:\\Users\\PUJA\\OneDrive\\Desktop\\Machine_Learning_Project\\ML_project-_using_MlFlow'

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: float
    criterion: str
    target_column: str

In [13]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            criterion = params.criterion,
            target_column = schema.name
            
        )

        return model_trainer_config


In [15]:
import pandas as pd
import os
from mlProject import logger
from sklearn.ensemble import RandomForestRegressor
import joblib


In [18]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]


        lr = RandomForestRegressor(n_estimators=self.config.n_estimators, criterion=self.config.criterion, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [19]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-01-11 14:41:52,358: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-11 14:41:52,365: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-11 14:41:52,372: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-11 14:41:52,380: INFO: common: created directory at: artifacts]
[2024-01-11 14:41:52,380: INFO: common: created directory at: artifacts/model_trainer]
